In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H3','D1']

# Load

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    BINS = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,3500,3800,4000,4200,4500,4800,5000,5200,5500,5800,6000]
    for x in BINS:  Calc[f'Chg Abs > {x}']      = (Calc['Change'].abs() > x).astype(int)
    for x in BINS:  Calc[f'Chg Abs > {x} +{1}'] =  Calc[f'Chg Abs > {x}'].shift(-1)
    for x in BINS:  Calc[f'Chg Abs > {x} +{2}'] =  Calc[f'Chg Abs > {x}'].shift(-2)
    for x in BINS:  Calc[f'Chg Abs > {x} +{3}'] =  Calc[f'Chg Abs > {x}'].shift(-3)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
def STATS(Calc):
    BINS = [100,200,500,800,1000,1200,1500,1800,2000,2200,2500,2800,3000,3200,3500,3800,4000,4200,4500,4800,5000,5200,5500,5800,6000]
    pipe = []
    for b in BINS:
        Df = Calc[Calc[f'Chg Abs > {b}']==1] .reset_index(drop=1)
        pipe.append({ 
            'Chg Abs > D0':  b,
            'Chg Abs > D+1': np.round(Df[f'Chg Abs > {b} +{1}'].mean() *100, 1), 
            'Chg Abs > D+2': np.round(Df[f'Chg Abs > {b} +{2}'].mean() *100, 1), 
            'Chg Abs > D+3': np.round(Df[f'Chg Abs > {b} +{3}'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

Sts = STATS(Calc)
Sts

,Chg Abs > D0,Chg Abs > D+1,Chg Abs > D+2,Chg Abs > D+3
0,100,95.0,95.0,95.3
1,200,89.6,89.4,89.8
2,500,75.1,75.5,76.7
3,800,60.8,65.2,63.5
4,1000,51.1,55.0,54.9
5,1200,46.8,49.9,48.1
6,1500,35.9,35.7,39.3
7,1800,27.2,28.2,29.4
8,2000,24.2,23.4,24.9
9,2200,23.1,20.5,21.8
